In [1]:
# Imports
from tensorflow.keras.layers import TextVectorization
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]

train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2,random_state=5)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
# Embedding setup
vectoriser = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectoriser.adapt(text_ds)

voc = vectoriser.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

glove_path = "glove.6B.100d.txt"
embeddings_index = {}
with open(glove_path,encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs,"f",sep= " ")
        embeddings_index[word] = coefs

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens,embedding_dim))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=keras.initializers.Constant(embedding_matrix),trainable=False)

In [4]:
# CNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.MaxPooling1D(1)(x)
x = layers.Conv1D(128, 1, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(units=1)(x)
cnn = keras.Model(int_sequences_input, preds)
cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [5]:
# LSTM setup

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.LSTM(4,input_shape=(1,5))(embedded_sequences)
out = layers.Dense(1,activation='relu')(x)

lstm = keras.Model(int_sequences_input,out)
lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [6]:
# RNN setup
int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.GRU(256,return_sequences=True)(embedded_sequences)
x = layers.SimpleRNN(128)(x)
out = layers.Dense(1,activation='relu')(x)

rnn = keras.Model(int_sequences_input,out)
rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])

In [7]:
# Data setup
x_train = vectoriser(np.array([[s] for s in train_samples])).numpy()
x_val = vectoriser(np.array([[s] for s in test_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

results = pd.DataFrame()

In [8]:
# Try CNN with batch sizes 10,32 and epochs 20,50,75 and 100
#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["CNN 20 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["CNN E20 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["CNN E100 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["CNN E100 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["CNN E75 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["CNN E75 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["CNN E50 B10"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

#cnn = keras.Model(int_sequences_input, preds)
#cnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
cnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["CNN E50 B32"] = cnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 5s 7ms/step - loss: 2.0984 - mean_absolute_error: 1.0927 - mean_absolute_percentage_error: 28.7434
Epoch 2/20
563/563 [==============================] - 4s 7ms/step - loss: 1.0184 - mean_absolute_error: 0.8075 - mean_absolute_percentage_error: 21.9927
Epoch 3/20
563/563 [==============================] - 5s 8ms/step - loss: 0.9682 - mean_absolute_error: 0.7895 - mean_absolute_percentage_error: 21.4204
Epoch 4/20
563/563 [==============================] - 5s 9ms/step - loss: 0.9064 - mean_absolute_error: 0.7634 - mean_absolute_percentage_error: 20.6946
Epoch 5/20
563/563 [==============================] - 5s 8ms/step - loss: 0.8618 - mean_absolute_error: 0.7381 - mean_absolute_percentage_error: 20.0744
Epoch 6/20
563/563 [==============================] - 5s 8ms/step - loss: 0.8450 - mean_absolute_error: 0.7376 - mean_absolute_percentage_error: 20.0360
Epoch 7/20
563/563 [==============================] - 5s 8ms/step - loss: 0.8335 -

In [9]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32
0,0.622289,0.696651,0.828249,0.825774,0.811100,0.807293,0.776588,0.792595
1,0.633850,0.666041,0.723627,0.718158,0.713910,0.712620,0.699339,0.707319
2,17.504826,17.723530,19.621828,19.421007,19.484289,19.251999,19.115721,19.453821


In [10]:
# Try LSTM with batch sizes 10,32 and epochs 20,50,75 and 100
#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=20)
results["LSTM E20 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=20)
results["LSTM E20 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["LSTM E100 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["LSTM E100 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["LSTM E75 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["LSTM E75 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["LSTM E50 B10"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

#lstm = keras.Model(int_sequences_input,out)
#lstm.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
lstm.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["LSTM E50 B32"] = lstm.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/20
563/563 [==============================] - 21s 35ms/step - loss: 17.0072 - mean_absolute_error: 3.9638 - mean_absolute_percentage_error: 97.1022
Epoch 2/20
563/563 [==============================] - 20s 36ms/step - loss: 11.7153 - mean_absolute_error: 3.2197 - mean_absolute_percentage_error: 76.9366
Epoch 3/20
563/563 [==============================] - 20s 35ms/step - loss: 8.2258 - mean_absolute_error: 2.6352 - mean_absolute_percentage_error: 61.1748
Epoch 4/20
563/563 [==============================] - 20s 36ms/step - loss: 6.4319 - mean_absolute_error: 2.2765 - mean_absolute_percentage_error: 51.7704
Epoch 5/20
563/563 [==============================] - 19s 34ms/step - loss: 5.0298 - mean_absolute_error: 1.9689 - mean_absolute_percentage_error: 44.2681
Epoch 6/20
563/563 [==============================] - 18s 33ms/step - loss: 3.9063 - mean_absolute_error: 1.6984 - mean_absolute_percentage_error: 38.0928
Epoch 7/20
563/563 [==============================] - 19s 33ms/step 

In [11]:
results

,CNN 20 B10,CNN E20 B32,CNN E100 B10,CNN E100 B32,CNN E75 B10,CNN E75 B32,CNN E50 B10,CNN E50 B32,LSTM E20 B10,LSTM E20 B32,LSTM E100 B10,LSTM E100 B32,LSTM E75 B10,LSTM E75 B32,LSTM E50 B10,LSTM E50 B32
0,0.622289,0.696651,0.828249,0.825774,0.811100,0.807293,0.776588,0.792595,1.259405,1.259522,1.259360,1.259465,1.259597,1.259444,1.259520,1.259575
1,0.633850,0.666041,0.723627,0.718158,0.713910,0.712620,0.699339,0.707319,0.935966,0.935918,0.935986,0.935942,0.935888,0.935950,0.935919,0.935896
2,17.504826,17.723530,19.621828,19.421007,19.484289,19.251999,19.115721,19.453821,28.358545,28.369537,28.354113,28.364195,28.376326,28.362194,28.369410,28.374420


In [12]:
# Try RNN with batch sizes 10,32 and epochs 50,75 and 100
#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=100)
results["RNN E100 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=100)
results["RNN E100 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=75)
results["RNN E75 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=75)
results["RNN E75 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=10,epochs=50)
results["RNN E50 B10"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

#rnn = keras.Model(int_sequences_input,out)
#rnn.compile(optimizer=Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=[tf.keras.losses.MeanAbsoluteError(),tf.keras.losses.MeanAbsolutePercentageError()])
rnn.fit(x=x_train,y=y_train,batch_size=32,epochs=50)
results["RNN E50 B32"] = rnn.evaluate(x=x_val,y=y_val,verbose=0)

Epoch 1/100
563/563 [==============================] - 72s 126ms/step - loss: 2.8225 - mean_absolute_error: 1.2909 - mean_absolute_percentage_error: 33.4905
Epoch 2/100
563/563 [==============================] - 73s 129ms/step - loss: 1.1284 - mean_absolute_error: 0.8354 - mean_absolute_percentage_error: 24.2677
Epoch 3/100
563/563 [==============================] - 71s 126ms/step - loss: 0.9858 - mean_absolute_error: 0.7878 - mean_absolute_percentage_error: 22.8508
Epoch 4/100
563/563 [==============================] - 71s 126ms/step - loss: 0.7446 - mean_absolute_error: 0.6898 - mean_absolute_percentage_error: 19.9703
Epoch 5/100
563/563 [==============================] - 71s 126ms/step - loss: 0.7259 - mean_absolute_error: 0.6816 - mean_absolute_percentage_error: 19.7105
Epoch 6/100
563/563 [==============================] - 71s 126ms/step - loss: 0.7491 - mean_absolute_error: 0.6930 - mean_absolute_percentage_error: 20.0664
Epoch 7/100
563/563 [==============================] - 71s

In [ ]:
results = results.T
#results.columns = ["MSE","MAE","% Error"]
results.to_csv("../../data/random_state_five/cnn_rnn_scores.csv")